In [2]:
EAGLE_PATH = r"D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/3_SegmentedAudios/master_manifest_PhilEagle.csv"
NOEAGLE_PATH = r"D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/3_SegmentedAudios/master_manifest_NoEagle.csv"

print("--- DIAGNOSTIC CHECK ---")

try:
    # Check Eagle File
    print(f"\n1. Checking: {EAGLE_PATH}")
    df1 = pd.read_csv(EAGLE_PATH)
    print(f"   Columns Found: {list(df1.columns)}")
    if 'location_id' in df1.columns:
        print("   ✅ 'location_id' exists!")
    else:
        print("   ❌ 'location_id' is MISSING.")

    # Check NoEagle File
    print(f"\n2. Checking: {NOEAGLE_PATH}")
    df2 = pd.read_csv(NOEAGLE_PATH)
    print(f"   Columns Found: {list(df2.columns)}")
    if 'location_id' in df2.columns:
        print("   ✅ 'location_id' exists!")
    else:
        print("   ❌ 'location_id' is MISSING.")

except Exception as e:
    print(f"\n❌ CRITICAL ERROR READING FILE: {e}")

--- DIAGNOSTIC CHECK ---

1. Checking: D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/3_SegmentedAudios/master_manifest_PhilEagle.csv
   Columns Found: ['start', 'end', 'label_base', 'label_full', 'quality', 'group_id', 'selection_numbers', 'location_id', 'segment_filename', 'output_folder', 'source_audio', 'label']
   ✅ 'location_id' exists!

2. Checking: D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/3_SegmentedAudios/master_manifest_NoEagle.csv
   Columns Found: ['start', 'end', 'label_base', 'label_full', 'quality', 'group_id', 'selection_numbers', 'location_id', 'segment_filename', 'output_folder', 'source_audio', 'label']
   ✅ 'location_id' exists!


In [ ]:
import os
import pandas as pd
import numpy as np
import librosa
import scipy.signal  # For Median Smoothing
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import (accuracy_score, f1_score, confusion_matrix, 
                             precision_recall_curve, average_precision_score, auc)
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.utils import resample
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# ===============================
# USER SETTINGS
# ===============================
# 1. Paths (Using your D:/ paths)
SEGMENTED_AUDIO_FOLDER = r"D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/3_SegmentedAudios"
MANIFEST_EAGLE = r"D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/3_SegmentedAudios/master_manifest_PhilEagle.csv"
MANIFEST_NOEAGLE = r"D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/3_SegmentedAudios/master_manifest_NoEagle.csv"
ANNOTATION_FOLDER = r"D:/_3rd Year Class/1st Sem/Machine Learning/_ForLE/For DataSet/2_SelectionTables"

# 2. Parameters
OVERLAP_THRESHOLD = 0.1 
MIN_SAMPLES_PER_SITE = 10 
PREDICTION_THRESHOLD = 0.30  # Lowered from 0.5 to catch more eagles
SMOOTHING_KERNEL = 3         # Window size for median filtering (3 segments)

# ===============================
# HELPER FUNCTIONS
# ===============================
def extract_audio_features(y, sr):
    if len(y) < sr * 0.5:
        y = np.pad(y, (0, int(sr * 0.5) - len(y)), mode='constant')
    
    features = []
    
    # 1. MFCCs (Static)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    features.append(np.mean(mfccs, axis=1))
    features.append(np.std(mfccs, axis=1))
    
    # 2. DELTA MFCCs (Rate of Change)
    mfcc_delta = librosa.feature.delta(mfccs)
    features.append(np.mean(mfcc_delta, axis=1))
    features.append(np.std(mfcc_delta, axis=1))
    
    # 3. DOUBLE DELTA (Acceleration)
    mfcc_delta2 = librosa.feature.delta(mfccs, order=2)
    features.append(np.mean(mfcc_delta2, axis=1))
    features.append(np.std(mfcc_delta2, axis=1))

    # 4. Spectral Features (Brightness/Pitch)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    features.append(np.mean(spec_cent))
    features.append(np.std(spec_cent))
    
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features.append(np.mean(spec_bw))
    
    spec_roll = librosa.feature.spectral_rolloff(y=y, sr=sr)
    features.append(np.mean(spec_roll))
    
    zcr = librosa.feature.zero_crossing_rate(y)
    features.append(np.mean(zcr))
    
    # 5. Spectral Contrast (Texture)
    spec_cont = librosa.feature.spectral_contrast(y=y, sr=sr)
    features.append(np.mean(spec_cont, axis=1))
    features.append(np.std(spec_cont, axis=1))
    
    return np.concatenate([np.array(f).flatten() for f in features])

def map_label(label_full):
    if "NoEagle" in label_full: return "NoEagle"
    return "Eagle"

def load_ground_truth_events(annotation_folder):
    gt_events = {}
    if not os.path.exists(annotation_folder): return {}
    for f in os.listdir(annotation_folder):
        if f.endswith(".txt"):
            base_name = f.split(".Table")[0].split(".txt")[0]
            path = os.path.join(annotation_folder, f)
            try:
                df = pd.read_csv(path, sep="\t", engine="python", comment="#")
                df.columns = [c.strip() for c in df.columns]
                if 'Type' in df.columns:
                     df = df[~df['Type'].astype(str).str.contains("necessar|ambiguous", case=False, na=False)]
                events = []
                for _, row in df.iterrows():
                    events.append({'start': float(row['Begin Time (s)']), 'end': float(row['End Time (s)'])} )
                gt_events[base_name] = events
            except: pass
    return gt_events

def calculate_bootstrap_ci(y_true, y_pred, n_iterations=1000, alpha=0.95):
    stats = []
    if len(y_true) < 10: return 0.0, 0.0
    for i in range(n_iterations):
        y_true_boot, y_pred_boot = resample(y_true, y_pred, random_state=i)
        if len(np.unique(y_true_boot)) < 2: continue
        score = f1_score(y_true_boot, y_pred_boot, pos_label="Eagle", average='binary')
        stats.append(score)
    if not stats: return 0.0, 0.0
    p = ((1.0 - alpha) / 2.0) * 100
    lower = max(0.0, np.percentile(stats, p))
    p = (alpha + ((1.0 - alpha) / 2.0)) * 100
    upper = min(1.0, np.percentile(stats, p))
    return lower, upper

# ===============================
# 1. LOAD & MERGE DATA (ROBUST)
# ===============================
print("📊 Loading Data...")

try:
    df_eagle = pd.read_csv(MANIFEST_EAGLE, encoding='utf-8-sig')
    df_noeagle = pd.read_csv(MANIFEST_NOEAGLE, encoding='utf-8-sig')
    
    # Force clean columns
    df_eagle.columns = [c.strip() for c in df_eagle.columns]
    df_noeagle.columns = [c.strip() for c in df_noeagle.columns]
    
    manifest = pd.concat([df_eagle, df_noeagle], ignore_index=True)
    
    # Fallback location check
    if 'location_id' not in manifest.columns:
        possible = [c for c in manifest.columns if 'loc' in c.lower()]
        if possible: manifest.rename(columns={possible[0]: 'location_id'}, inplace=True)
        else:
            print("❌ Critical: No location column found.")
            exit()

    # FILTER TINY SITES
    site_counts = manifest['location_id'].value_counts()
    valid_sites = site_counts[site_counts >= MIN_SAMPLES_PER_SITE].index.tolist()
    manifest = manifest[manifest['location_id'].isin(valid_sites)]
    
    print(f"   Total Valid Samples: {len(manifest)} (Filtered sites < {MIN_SAMPLES_PER_SITE} samples)")

except Exception as e:
    print(f"❌ Error loading manifests: {e}")
    exit()

gt_data = load_ground_truth_events(ANNOTATION_FOLDER)

all_features = []
all_labels = []
all_groups = [] 
all_metadata = []   

print("\n🎧 Extracting Features (Upgraded)...")
for idx, row in tqdm(manifest.iterrows(), total=len(manifest)):
    filename = row['segment_filename']
    label_full = row.get('label_full', row['label'])
    
    # Path Logic
    if "NoEagle" in label_full:
        folder_path = "NoEagleAudio" 
    else:
        location = row['location_id']
        label_folder = label_full 
        folder_path = os.path.join(location, label_folder)
        
    audio_path = os.path.join(SEGMENTED_AUDIO_FOLDER, folder_path, filename)
    
    if os.path.exists(audio_path):
        try:
            y, sr = librosa.load(audio_path, sr=None)
            feat = extract_audio_features(y, sr)
            lbl = map_label(row.get('label_base', row['label']))
            
            all_features.append(feat)
            all_labels.append(lbl)
            all_groups.append(row['location_id'])
            
            all_metadata.append({
                'source_audio': row['source_audio'], 
                'start_time': row['segment_start_time'],
                'end_time': row['segment_end_time']
            })
        except: pass

X = np.array(all_features)
y = np.array(all_labels)
groups = np.array(all_groups)
metadata = np.array(all_metadata)

le = LabelEncoder()
y_encoded = le.fit_transform(y)
if "Eagle" in le.classes_:
    eagle_idx = le.transform(["Eagle"])[0]
else:
    print("❌ Error: No 'Eagle' class found in data!")
    exit()

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ===============================
# 2. LOSO LOOP (CRASH-PROOF)
# ===============================
print("\n" + "="*50)
print("🚀 STARTING ROBUST LOSO VALIDATION")
print("="*50)

# Check Global Balance first
if len(np.unique(y_encoded)) < 2:
    print("❌ CRITICAL: Loaded dataset has only 1 class. Check paths!")
    exit()

logo = LeaveOneGroupOut()
site_metrics = []

y_true_global = []
y_pred_global = []
y_prob_global = []

for train_idx, test_idx in logo.split(X_scaled, y_encoded, groups=groups):
    test_site = groups[test_idx][0]
    
    if test_site == "GeneralForest":
        continue 

    print(f"\n📍 Testing Site: {test_site}")
    
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y_encoded[train_idx], y_encoded[test_idx]
    meta_test = metadata[test_idx] 
    
    # --- CRASH FIX: CHECK SPLIT BALANCE ---
    if len(np.unique(y_train)) < 2:
        print(f"   ⚠️ SKIPPING: Training set for {test_site} has only 1 class.")
        continue

    # Train
    rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1, class_weight='balanced')
    rf.fit(X_train, y_train)
    
    # Predict (Threshold Moving)
    y_prob = rf.predict_proba(X_test)[:, eagle_idx]
    y_pred = (y_prob >= PREDICTION_THRESHOLD).astype(int) 
    
    # Metrics
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, pos_label=eagle_idx, average='binary')
    try: ap = average_precision_score(y_test, y_prob, pos_label=eagle_idx)
    except: ap = 0.0
    
    # Event Reconstruction (Median Smoothing)
    files_to_process = {}
    for i, m in enumerate(meta_test):
        fname = m['source_audio'].replace('.wav', '')
        if fname not in files_to_process: files_to_process[fname] = []
        is_eagle = (y_pred[i] == eagle_idx)
        files_to_process[fname].append({'start': m['start_time'], 'end': m['end_time'], 'is_eagle': is_eagle})
    
    tp, fp, fn = 0, 0, 0
    for fname, segments in files_to_process.items():
        segments.sort(key=lambda x: x['start'])
        
        # Apply Smoothing
        raw_preds = [1 if s['is_eagle'] else 0 for s in segments]
        if len(raw_preds) >= SMOOTHING_KERNEL:
            smoothed = scipy.signal.medfilt(raw_preds, kernel_size=SMOOTHING_KERNEL)
        else:
            smoothed = raw_preds
            
        for k, val in enumerate(smoothed):
            segments[k]['is_eagle'] = (val == 1)

        pred_events = []
        curr = None
        for seg in segments:
            if seg['is_eagle']:
                if curr and (seg['start'] - curr['end'] < 0.1): curr['end'] = seg['end']
                else: 
                    if curr: pred_events.append(curr)
                    curr = {'start': seg['start'], 'end': seg['end']}
            else:
                if curr: pred_events.append(curr); curr = None
        if curr: pred_events.append(curr)
        
        real_events = gt_data.get(fname, [])
        matched = set()
        for p in pred_events:
            hit = False
            for i_gt, r in enumerate(real_events):
                intersection = max(0, min(p['end'], r['end']) - max(p['start'], r['start']))
                coverage = intersection / (r['end'] - r['start']) if (r['end'] - r['start']) > 0 else 0
                if coverage >= OVERLAP_THRESHOLD: hit = True; matched.add(i_gt); break
            if hit: tp += 1
            else: fp += 1
        fn += len(real_events) - len(matched)
        
    prec_event = tp/(tp+fp) if (tp+fp)>0 else 0
    rec_event = tp/(tp+fn) if (tp+fn)>0 else 0
    f1_event = 2*(prec_event*rec_event)/(prec_event+rec_event) if (prec_event+rec_event)>0 else 0

    print(f"   🔹 Seg F1: {f1:.3f} | Event F1: {f1_event:.3f} | AP: {ap:.3f}")
    
    site_metrics.append({
        'site': test_site, 
        'seg_acc': acc, 'seg_f1': f1, 'seg_ap': ap,
        'event_f1': f1_event,
        'samples': len(y_test)
    })
    
    y_true_global.extend(le.inverse_transform(y_test))
    y_pred_global.extend(le.inverse_transform(y_pred)) 
    y_prob_global.extend(y_prob)

# ===============================
# 3. REPORTING
# ===============================
print("\n" + "="*50)
print("📊 FINAL FEASIBILITY REPORT (OPTIMIZED)")
print("="*50)

if not site_metrics:
    print("❌ No valid sites.")
else:
    df_res = pd.DataFrame(site_metrics)
    print(df_res[['site', 'samples', 'seg_f1', 'event_f1']])
    print("-" * 30)

    # Global Weighted Averages
    global_f1 = f1_score(y_true_global, y_pred_global, pos_label="Eagle", average='binary')
    global_acc = accuracy_score(y_true_global, y_pred_global)
    global_ap = average_precision_score(
        [1 if y == "Eagle" else 0 for y in y_true_global], 
        y_prob_global
    )

    print(f"🏆 Global Weighted F1:       {global_f1:.3f}")
    print(f"🏆 Global Accuracy:          {global_acc:.2%}")
    print(f"🏆 Global AP (PR Area):      {global_ap:.3f}")

    print("\n🔄 Calculating Bootstrap 95% CI...")
    lower, upper = calculate_bootstrap_ci(y_true_global, y_pred_global)
    print(f"   Confidence Interval: [{lower:.3f}, {upper:.3f}]")

    cm = confusion_matrix(y_true_global, y_pred_global, labels=["Eagle", "NoEagle"])
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=["Eagle", "NoEagle"], yticklabels=["Eagle", "NoEagle"])
    plt.title(f'Global Confusion Matrix\n(Optimized F1: {global_f1:.2f})')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    precision, recall, _ = precision_recall_curve(
        [1 if y == "Eagle" else 0 for y in y_true_global], 
        y_prob_global
    )
    pr_auc = auc(recall, precision)

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='darkorange', lw=2, label=f'PR curve (AP = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Global Precision-Recall Curve')
    plt.legend(loc="lower left")
    plt.grid(True, alpha=0.3)
    plt.show()

📊 Loading Data...
   Total Valid Samples: 2975 (Filtered sites < 10 samples)

🎧 Extracting Features (Upgraded)...


100%|██████████| 2975/2975 [06:28<00:00,  7.67it/s]



🚀 STARTING IMPROVED LOSO VALIDATION

📍 Testing Site: Bukidnon


IndexError: index 1072 is out of bounds for axis 0 with size 0